# Landscape evolution model, steady-state and beat effect

See https://fastscape.org/

In [ ]:
import ipytone
import numpy as np
import matplotlib.pyplot as plt
import xsimlab as xs

from fastscape.models import basic_model
from ipyfastscape import TopoViz3d

Let's run a 3 millions years "classic" landscape evolution simulation, where uniform block uplift competes with river channel erosion and hillslope erosion (diffusion).

In [ ]:
in_ds = xs.create_setup(
    model=basic_model,
    clocks={
        'clock': np.arange(0, 3e6 + 2e4, 2e4),
        'time': np.arange(0, 3e6 + 2e4, 2e4),
    },
    master_clock='clock',
    input_vars={
        'grid__shape': [201, 201],
        'grid__length': [2e5, 2e5],
        'boundary__status': 'fixed_value',
        'uplift__rate': 1e-3,
        'spl': {
            'k_coef': 1e-6,
            'area_exp': 0.6,
            'slope_exp': 1
        },
        'diffusion__diffusivity': 1e-3
    },
    output_vars={
        'topography__elevation': 'time',
        'erosion__rate': 'time',
    }
)

In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=basic_model)

Let's compute spatially averaged total erosion rates.

In [ ]:
out_ds["erosion__rate_avg"] = out_ds.erosion__rate.mean(('x', 'y'))

Let's compare it with the uplift rate in a plot. The erosion rate should converge towards the uplift rate (steady-state landscape).

In [ ]:
fig, ax = plt.subplots()

out_ds.erosion__rate_avg.plot(ax=ax);
plt.hlines([out_ds.uplift__rate], xmin=0, xmax=3e6, color="r");

## Sonification

Create two oscillators: one for the uplift rate and another for the erosion rate. Those are slightly panned on the left and right channels.

In [ ]:
pan_uplift = ipytone.Panner(pan=-0.2)
osc_uplift = ipytone.Oscillator(volume=-7).chain(pan_uplift, ipytone.destination)

pan_erosion = ipytone.Panner(pan=0.2)
osc_erosion = ipytone.Oscillator(volume=-7).chain(pan_erosion, ipytone.destination)

Convert uplift / erosion rates into oscillator frequencies

In [ ]:
rate2freq_factor = 1e6

uplift_freq = float(out_ds.uplift__rate * rate2freq_factor)

erosion_freq = out_ds.erosion__rate_avg * rate2freq_factor

# avoid too low frequencies
erosion_freq = np.where(erosion_freq < 50, 50, erosion_freq)

# small offset to hear the beat interference
erosion_freq += 20

Initial frequencies

In [ ]:
osc_uplift.frequency.value = uplift_freq
osc_erosion.frequency.value = erosion_freq[0]

## Terrain 3D visualization

In [ ]:
app = TopoViz3d(out_ds, canvas_height=600, time_dim="time")

app.components['vertical_exaggeration'].set_factor(8)

In [ ]:
def change_osc_freq(change):
    osc_erosion.frequency.ramp_to(erosion_freq[change["new"]], 0.1)
                                               
app.components['timestepper'].slider.observe(change_osc_freq, names='value')

In [ ]:
app.show()

In [ ]:
osc_erosion.start()
osc_uplift.start()

In [ ]:
osc_erosion.stop()
osc_uplift.stop()

In [ ]:
osc_erosion.dispose()
osc_uplift.dispose()